In [12]:
#!pip install scikit-learn
#!pip install casatools casatasks


In [1]:
import scipy
import pylab
import os
import gc
import pprint
import numpy as np
import pandas as pd
import csv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.io.fits as fits
from astropy import units as u
from astropy.wcs import WCS
import pyspeckit
import analysis as an
import pyneb as pn
from spectral_cube import SpectralCube 
import sklearn
from sklearn.decomposition import PCA
from reproject import reproject_interp
from astropy.convolution import convolve, Gaussian2DKernel



%matplotlib inline
                                            # Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

                                            # LaTeX:
matplotlib.rc('text', usetex=True)
#matplotlib.rc('font', family='sans-serif')
font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)
#if not working: in terminal where launch jupyter notebook run
#export PATH=$PATH:/Library/TeX/texbin/latex

## CASA imports

In [2]:
from casatools import image, table
from casatasks import importfits, imstat, tclean, exportfits
from casatools import image, regionmanager
from casatasks import imcontsub

import os
import csv
import shutil
from glob import glob

# CASA tasks (assumed to be available in your environment)
from casatasks import importfits, imcontsub, exportfits

data_update using version casarundata-2025.12.12-1.tar.gz, acquiring the lock ... 


lock file is not empty : /Users/u826s018/.casa/data/data_update.lock
data_update: the lock file at /Users/u826s018/.casa/data is not empty.
A previous attempt to update path may have failed or exited prematurely.
Remove the lock file and set force to True with the desired version (default to most recent).
It may be best to completely re-populate path using pull_data and measures_update.
2026-02-07 16:58:29	SEVERE	::casa	lock file is not empty : /Users/u826s018/.casa/data/data_update.lock
2026-02-07 16:58:29	SEVERE	::casa	data_update: the lock file at /Users/u826s018/.casa/data is not empty.
2026-02-07 16:58:29	SEVERE	::casa	A previous attempt to update path may have failed or exited prematurely.
2026-02-07 16:58:29	SEVERE	::casa	Remove the lock file and set force to True with the desired version (default to most recent).
2026-02-07 16:58:29	SEVERE	::casa	It may be best to completely re-populate path using pull_data and measures_update.
2026-02-07 16:58:29	SEVERE	::casa	
2026-02-07 16:5

In [3]:
# This block creates the initial parameter CSV for continuum subtraction.
# It only generates entries for files that do NOT already have a contsub
# parameter entry in the folder. This is meant for the first manual pass.
#
# Workflow:
# 1. First, run this cell to generate a new parameter CSV containing the
#    channel values you determined manually for each line/cube.
#
# 2. After inspecting the individual channels and updating the values in
#    this new CSV, you should merge it into the main parameter file using
#    the merge script in the next code block.
#
# 3. Once the contsub file exists, you normally do NOT rerun this cell.
#    Instead, you simply edit the CSV directly and run the autolooping code.
#
# 4. If you ever need to rerun the workflow for a single line without
#    overwriting everything, you can delete that one entry, regenerate it
#    with this cell, update it manually, and then merge it again.
#
# Requirements:
# - The input_dir must contain the line subcubes.
# - CASA will write the continuum-subtracted outputs to output_dir.


input_dir = "../../github/astr796_25_V2/NGC253_output/3channel/atomic"
output_dir = "../../github/astr796_25_V2/NGC253_output/3channel/casa/atomic"
os.makedirs(output_dir, exist_ok=True)

log_file = os.path.join(output_dir, "imcontsub_params.csv")

# Load existing processed line names
processed_lines = set()
if os.path.exists(log_file):
    with open(log_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        line_key = "line_name" if "line_name" in header else header[0]  # fallback to first column
        for row in reader:
            processed_lines.add(row[line_key])


# Prepare to collect new entries
new_entries = []

fits_files = glob(os.path.join(input_dir, "*.fits"))
for fits_path in fits_files:
    line_name = os.path.splitext(os.path.basename(fits_path))[0]
    casa_image_path = os.path.join(output_dir, f"{line_name}.im")
    linefile = os.path.join(output_dir, f"{line_name}.line")
    contfile = os.path.join(output_dir, f"{line_name}.cont")

    # Skip if already processed or output files exist
    if line_name in processed_lines or (os.path.exists(linefile) and os.path.exists(contfile)):
        print(f"Skipping {line_name} (already processed or output files exist)")
        continue

    # Import FITS to CASA
    importfits(
        fitsimage=fits_path,
        imagename=casa_image_path,
        whichhdu=0,
        whichrep=0,
        zeroblanks=True,
        overwrite=True,
        defaultaxes=False,
        defaultaxesvalues=[],
        beam=[]
    )

    # Ask user for parameters
    chans = input(f"Enter chans value for {line_name}: ")
    fitorder = int(input(f"Enter fitorder for {line_name} (e.g., 0, 1, 2): "))

    # Run imcontsub
    imcontsub(
        imagename=casa_image_path,
        linefile=linefile,
        contfile=contfile,
        fitorder=fitorder,
        region='',
        box='',
        chans=chans,
        stokes=''
    )

    # Export to FITS
    exportfits(imagename=linefile, fitsimage=linefile + ".fits", overwrite=True)
    exportfits(imagename=contfile, fitsimage=contfile + ".fits", overwrite=True)

    # Store new entry
    new_entries.append([line_name, fitorder, chans, casa_image_path, linefile, contfile])

# Append new entries to CSV, writing header only if needed
write_header = not os.path.exists(log_file) or os.path.getsize(log_file) == 0

with open(log_file, "a", newline="") as f:
    writer = csv.writer(f)
    if write_header:
        writer.writerow(["line_name", "fitorder", "chans", "imagename", "linefile", "contfile"])
    for entry in new_entries:
        writer.writerow(entry)


print(f"Appended {len(new_entries)} new entries to:", log_file)


Skipping NGC253_CO2_15.0 (already processed or output files exist)
Skipping NGC253_MgV_5.6 (already processed or output files exist)
Skipping NGC253_H5b_7.5 (already processed or output files exist)
Skipping NGC253_H6d_5.1 (already processed or output files exist)
Skipping NGC253_H7z_6.3 (already processed or output files exist)
Skipping NGC253_ArII_7.0 (already processed or output files exist)
Skipping NGC253_He_6.7 (already processed or output files exist)
Skipping NGC253_H7k_5.4 (already processed or output files exist)
Skipping NGC253_MgV_13.5 (already processed or output files exist)
Skipping NGC253_H6g_5.9 (already processed or output files exist)
Skipping NGC253_NeIII_15.6 (already processed or output files exist)
Skipping NGC253_H7e_6.8 (already processed or output files exist)
Skipping NGC253_NeII_12.8 (already processed or output files exist)
Skipping NGC253_H6a_12.4 (already processed or output files exist)
Skipping NGC253_H8b_16.2 (already processed or output files exist)
S

In [5]:
'''
# This script is used to update the main parameter file used by the autolooping code.
# The workflow is:
# 1. For each new line or cube, we first inspect the individual channels manually
#    and record the correct channel values in a separate parameter file (new_file).
#
# 2. The autolooping code always reads from the main parameter file (main_file),
#    so before running the autoloop we merge the updated entries from new_file
#    into main_file.
#
# 3. This script performs that merge: it combines both files, keeps the most recent
#    entry for each line_name, and writes the updated result back to main_file.
#
# 4. If we need to rerun the workflow for only one specific line without modifying
#    the main parameter file, we can create a temporary parameter file, update only
#    that line there, and then run this merge script to safely update main_file.
#
# This keeps the main parameter file clean and ensures that autolooping always uses
# the latest manually verified channel values.

import pandas as pd

main_file = "../NGC253_output/3channel/imcontsub_params.csv"
new_file = "../NGC253_output/3channel/casa/atomic/imcontsub_params.csv"

df_main = pd.read_csv(main_file)
df_new = pd.read_csv(new_file)

df_new.columns = df_main.columns  # ensure column match

df_merged = pd.concat([df_main, df_new], ignore_index=True)
df_merged.drop_duplicates(subset="line_name", keep="last", inplace=True)  # keep updated rows

df_merged.to_csv(main_file, index=False)

print(" Merged and updated with replacements:", main_file)
'''

'\n# This script is used to update the main parameter file used by the autolooping code.\n# The workflow is:\n# 1. For each new line or cube, we first inspect the individual channels manually\n#    and record the correct channel values in a separate parameter file (new_file).\n#\n# 2. The autolooping code always reads from the main parameter file (main_file),\n#    so before running the autoloop we merge the updated entries from new_file\n#    into main_file.\n#\n# 3. This script performs that merge: it combines both files, keeps the most recent\n#    entry for each line_name, and writes the updated result back to main_file.\n#\n# 4. If we need to rerun the workflow for only one specific line without modifying\n#    the main parameter file, we can create a temporary parameter file, update only\n#    that line there, and then run this merge script to safely update main_file.\n#\n# This keeps the main parameter file clean and ensures that autolooping always uses\n# the latest manually ve

### Autolooping code

In [4]:
#Advanced form of autolooping code should be able to handle errors or any issues
# if doesnot work go to last to find initial version to tackle problem
#Will use the parameter file csv and run all deleting fits file before creating line and continuum automatically


# Define directories
input_dir = "../../github/astr796_25_V2/NGC253_output/3channel/atomic"
output_dir = "../../github/astr796_25_V2/NGC253_output/3channel/casa/atomic"
os.makedirs(output_dir, exist_ok=True)

log_file = os.path.join(output_dir, "imcontsub_params.csv")
param_file = os.path.join(input_dir, "imcontsub_params.csv")

# Clean output_dir before starting
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Load parameters from CSV into a dictionary
params = {}
with open(param_file, "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        line_name = row["line_name"].strip()
        chans = row["chans"].strip()
        fitorder_str = row["fitorder"].strip()

        if fitorder_str in ("1", "2"):
            fitorder = int(fitorder_str)
            params[line_name] = {"chans": chans, "fitorder": fitorder}
        else:
            print(f"Skipping '{line_name}': invalid or missing fitorder '{fitorder_str}'")

# Initialize CSV logging if it doesn't exist
if not os.path.exists(log_file):
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["line_name", "fitorder", "chans", "imagename", "linefile", "contfile"])

# Loop through all FITS files in input directory
fits_files = glob(os.path.join(input_dir, "*.fits"))

for fits_path in fits_files:
    line_name = os.path.splitext(os.path.basename(fits_path))[0]

    if line_name not in params:
        print(f" Skipping {line_name}: no parameters found in {param_file}")
        continue

    chans = params[line_name]["chans"]
    fitorder = params[line_name]["fitorder"]

    casa_image_path = os.path.join(output_dir, f"{line_name}.im")
    linefile = os.path.join(output_dir, f"{line_name}.line")
    contfile = os.path.join(output_dir, f"{line_name}.cont")

    # Import FITS to CASA
    importfits(
        fitsimage=fits_path,
        imagename=casa_image_path,
        whichhdu=0,
        whichrep=0,
        zeroblanks=True,
        overwrite=True,
        defaultaxes=False,
        defaultaxesvalues=[],
        beam=[]
    )

    # Run imcontsub
    imcontsub(
        imagename=casa_image_path,
        linefile=linefile,
        contfile=contfile,
        fitorder=fitorder,
        region='',
        box='',
        chans=chans,
        stokes=''
    )

    # Export line and continuum to FITS
    exportfits(imagename=linefile, fitsimage=linefile + ".fits", overwrite=True)
    exportfits(imagename=contfile, fitsimage=contfile + ".fits", overwrite=True)

    # Record parameters into run log
    with open(log_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([line_name, fitorder, chans, casa_image_path, linefile, contfile])

print("All files processed. Run log saved to:", log_file)


2026-02-07 16:59:27	WARN	importfits::::casa	This image has no beam or angular resolution provided, so you will not receive warnings from
2026-02-07 16:59:27	WARN	importfits::::casa+	tasks such as imregrid if your image pixels do not sample the the angular resolution well.
2026-02-07 16:59:27	WARN	importfits::::casa+	(This only affects warnings, not any functionality).
2026-02-07 16:59:27	WARN	importfits::::casa+	Providing a beam and brightness units in an image can also be useful for flux calculations.
2026-02-07 16:59:27	WARN	importfits::::casa+	If you wish to add a beam or brightness units to your image, please use
2026-02-07 16:59:27	WARN	importfits::::casa+	the "beam" parameter or ia.setrestoringbeam() and ia.setbrightnessunit()
2026-02-07 16:59:27	WARN	importfits::::casa	This image has no beam or angular resolution provided, so you will not receive warnings from
2026-02-07 16:59:27	WARN	importfits::::casa+	tasks such as imregrid if your image pixels do not sample the the angular r

All files processed. Run log saved to: ../../github/astr796_25_V2/NGC253_output/3channel/casa/atomic/imcontsub_params.csv


## Smoothing and regriding the line file

## Code I might require for checks/ error in workflow, these may contain my earlier version of codes

### Sanity Check